<a href="https://colab.research.google.com/github/mgp87/Jupyter_Notebooks_Collection/blob/main/Jamba_Inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -qqq transformers>=4.39.0 mamba-ssm causal-conv1d>=1.2.0 accelerate bitsandbytes --progress-bar off
!pip install flash-attn --no-build-isolation

import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

# Load model in 8-bit precision
quantization_config = BitsAndBytesConfig(
    load_in_8bit=True,
    llm_int8_skip_modules=["mamba"]
)
model = AutoModelForCausalLM.from_pretrained(
    "ai21labs/Jamba-v0.1",
    trust_remote_code=True,
    torch_dtype=torch.bfloat16,
    attn_implementation="flash_attention_2",
    quantization_config=quantization_config
)
tokenizer = AutoTokenizer.from_pretrained("ai21labs/Jamba-v0.1")

In [ ]:
# Tokenize input
prompt = """George wants to warm his hands quickly by rubbing them. Which skin surface will produce the most heat? A. dry palms B. wet palms C. palms covered with oil D. palms covered with lotion. Answer:"""
input_ids = tokenizer(
    prompt,
    return_tensors='pt'
).to(model.device)["input_ids"]

# Generate answer
outputs = model.generate(input_ids, max_new_tokens=216)

# Print output
print(tokenizer.batch_decode(outputs))